In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_26141/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [5]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
16,model_15_9_7,0.991173,0.991590,0.931537,0.980337,0.009768,0.013732,0.033699,0.023128,"Hidden Size=[40], regularizer=0.01, learning_r..."
17,model_15_9_6,0.991134,0.992912,0.935147,0.982020,0.009812,0.011573,0.031922,0.021149,"Hidden Size=[40], regularizer=0.01, learning_r..."
18,model_15_9_8,0.991116,0.990285,0.928075,0.978697,0.009833,0.015862,0.035403,0.025058,"Hidden Size=[40], regularizer=0.01, learning_r..."
20,model_15_9_5,0.990963,0.994220,0.938871,0.983715,0.010002,0.009437,0.030089,0.019155,"Hidden Size=[40], regularizer=0.01, learning_r..."
23,model_15_9_4,0.990613,0.995470,0.942655,0.985379,0.010389,0.007396,0.028226,0.017198,"Hidden Size=[40], regularizer=0.01, learning_r..."
...,...,...,...,...,...,...,...,...,...,...
719,model_10_7_0,0.967410,0.986826,0.969192,0.976933,0.036067,0.022833,0.078421,0.048992,"Hidden Size=[25], regularizer=0.1, learning_ra..."
725,model_13_9_4,0.966993,0.950758,0.953385,0.953853,0.036529,0.066321,0.035457,0.051797,"Hidden Size=[35], regularizer=0.01, learning_r..."
727,model_10_7_8,0.966809,0.940965,0.955008,0.949122,0.036732,0.102315,0.114526,0.108061,"Hidden Size=[25], regularizer=0.1, learning_ra..."
737,model_10_9_0,0.966434,0.964453,0.995687,0.988023,0.037147,0.026124,0.007149,0.017195,"Hidden Size=[25], regularizer=0.1, learning_ra..."
